In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [95]:
train = pd.read_csv("/kaggle/input/iris-type-classification/train.csv")
test = pd.read_csv("/kaggle/input/iris-type-classification/test.csv")

In [96]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

features = ['sepal_length_(cm)', 'sepal_width_(cm)',
            'petal_length_(cm)', 'petal_width_(cm)']

scaler = StandardScaler()

scaler.fit(train[features])
train[features] = scaler.transform(train[features])
test[features] = scaler.transform(test[features])

X, y = train[features], train['target']
X_test = test[features]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42, test_size=0.2)

In [97]:
from catboost import Pool, CatBoostClassifier, EShapCalcType, EFeaturesSelectionAlgorithm

train_pool = Pool(X_train, y_train)
valid_pool = Pool(X_valid, y_valid)

params = {
    'loss_function': 'MultiClass',
    'max_depth': 2,
    'eval_metric': 'TotalF1:average=Macro',
    'iterations': 1,
    'metric_period': 1,
    'random_seed': 42,
    'classes_count': 3,
}

model = CatBoostClassifier(**params)

# summary = model.select_features(
#     train_pool,
#     eval_set=valid_pool,
#     features_for_select=features,
#     num_features_to_select=2,
#     steps=3,
#     algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
#     shap_calc_type=EShapCalcType.Regular,
#     train_final_model=True,
#     logging_level='Silent',
#     plot=True
# )

model.fit(train_pool, eval_set=valid_pool)

Learning rate set to 0.5
0:	learn: 0.9377953	test: 0.9581699	best: 0.9581699 (0)	total: 353us	remaining: 0us

bestTest = 0.9581699346
bestIteration = 0



In [98]:
from sklearn.metrics import f1_score

score = f1_score(y_valid, model.predict(X_valid).flatten(), average='macro')

print(f"Macro F1 Score: {score:.5f}")

Macro F1 Score: 0.95817


In [99]:
y_pred = model.predict(X_test)

subm = pd.DataFrame({
    'SampleID': test['SampleID'],
    'label': y_pred.flatten().tolist()
})

subm.head()

,SampleID,label
0,39,0
1,128,2
2,58,1
3,94,1
4,43,0


In [100]:
subm.to_csv("submission.csv", index=False)